In [18]:
import pandas as pd
import json
from pymongo import MongoClient

In [ ]:
def transform_csv_to_json(cvs_file:str):
    # Read the csv
    df = pd.read_csv(cvs_file)
    # clean and transform the data => for example, remove white spaces from column names and convert them to lowercase

    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    
    # TO DO Convert to expected data types
    
   # TO DO create a sector_columns list with all the columns that are not year, gdp, urban_population, country
    
    
    sector_columns = []
    
    # Convert sector columns to float
    for col in sector_columns:
        df[col] = df[col].astype(float)
    
   # TO DO Replace all the NaN values with "Null"

    # Group by country and create nested structure
    result = (
        df.groupby("country")
        .apply(lambda group: {
            "country": group["country"].iloc[0],
            "years": group.apply(lambda row: {
                row["year"]:{
                "gdp": row["gdp"],
                "urban_population": row["urban_population"],
                "sectors": {col: row[col] for col in sector_columns}
                }
            }, axis=1).tolist()
        })
        .tolist()
    )

    return result



In [20]:

csv_file = "pay_gap_Europe.csv"  

transformed_data = transform_csv_to_json(csv_file)

# Print first document as a sample
print(json.dumps(transformed_data[:1], indent=4))

[
    {
        "country": "Austria",
        "years": [
            {
                "2010": {
                    "gdp": 35390.0,
                    "urban_population": 57.4,
                    "sectors": {
                        "industry": 24.0,
                        "business": 25.2,
                        "mining": 18.3,
                        "manufacturing": 24.4,
                        "electricity_supply": 23.6,
                        "water_supply": 12.2,
                        "construction": 9.9,
                        "retail_trade": 27.5,
                        "transportation": 7.3,
                        "accommodation": 9.9,
                        "information": 21.2,
                        "financial": 30.3,
                        "real_estate": 27.0,
                        "professional_scientific": 34.0,
                        "administrative": 22.5,
                        "public_administration": "Null",
                        "education": 27.

/var/folders/wc/4r8t_c91523cgt8st98mw_ph0000gn/T/ipykernel_719/4156298017.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: {


In [21]:
    # Save as JSON file
with open("output.json", "w", encoding="utf-8") as f:
        json.dump(transformed_data, f, indent=4, ensure_ascii=False)

In [ ]:


def upload_to_mongodb(json_file, db_name="paydata", collection_name="countries"):
    """
    Uploads the transformed JSON data into MongoDB.

    Parameters:
        json_file (str): Path to the JSON file containing the transformed data.
        db_name (str): MongoDB database name.
        collection_name (str): MongoDB collection name.
    
    Returns:
        int: Number of inserted documents.
    """

    # Connect to MongoDB (Modify connection string if needed)
    db_string =""

    client = MongoClient(db_string)  # Change if using a remote server
    db = client[db_name]
    collection = db[collection_name]

    # TO DO Load JSON file


    # Insert or update documents (upsert to avoid duplicates)
    for doc in data:
        collection.update_one(
             {"country": doc["country"]},# Match existing country
            {"$set": doc},  # Update or insert data
            upsert=True  # Create if doesn't exist
        )

    print(f"Inserted/Updated {len(data)} documents into MongoDB.")
    return len(data)



In [23]:
upload_to_mongodb("output.json","PayGapDB","PayGapCollection")

Inserted/Updated 27 documents into MongoDB.


27